### Import

In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

### GPU

In [3]:
torch.cuda.device_count()

cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')

device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

### INIT

In [4]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Mono'
excl_folders = ['NoFX_mono', 'NoFX_mono_preprocessed']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=6

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, shuffle=True)
# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

In [5]:
len(dataset)

163488

### TRAIN FxNET and SetNET

In [6]:
# model
fxnet = models.FxNet().to(device)
# optimizer
optimizer_fxnet = optim.Adam(fxnet.parameters(), lr=0.001)
# loss function
loss_func_fxnet = nn.CrossEntropyLoss()

# model
setnet = models.SettingsNet().to(device)
# optimizer
optimizer_setnet = optim.Adam(setnet.parameters(), lr=0.001)
# loss function
loss_func_setnet = nn.MSELoss(reduction='mean')

In [7]:
print(fxnet)
print(setnet)

FxNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=14, bias=True)
)
SettingsNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=6, bias=True)
)


In [8]:
# TRAIN and TEST FxNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses_fx, all_val_losses_fx, all_test_losses_fx = [],[],[]
all_train_correct_fx, all_val_correct_fx, all_test_correct_fx = [],[],[]
all_train_results_fx, all_val_results_fx, all_test_results_fx = [],[],[]

all_train_losses_set, all_val_losses_set, all_test_losses_set = [],[],[]
all_train_correct_set, all_val_correct_set, all_test_correct_set = [],[],[]
all_train_results_set, all_val_results_set, all_test_results_set = [],[],[]

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainer.train_fx_net(
        model=fxnet,
        optimizer=optimizer_fxnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_fx_net(
        model=fxnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler, 
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_fx_net(
        model=fxnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler, 
        device=device
    )

    all_train_losses_fx.append(train_loss)
    all_val_losses_fx.append(val_loss)
    all_test_losses_fx.append(test_loss)
    
    all_train_correct_fx.append(train_correct)
    all_val_correct_fx.append(val_correct)
    all_test_correct_fx.append(test_correct)
    
    all_train_results_fx.append(train_results)
    all_val_results_fx.append(val_results)
    all_test_results_fx.append(test_results)

    train_loss, train_correct, train_results = trainvaltest.train_settings_net(
        model=setnet,
        optimizer=optimizer_setnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch,
        loss_function=loss_func_setnet, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainvaltest.val_settings_net(
        model=setnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler,
        loss_function=loss_func_setnet,
        device=device
    )
    
    test_loss, test_correct, test_results = trainvaltest.test_settings_net(
        model=setnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler,
        loss_function=loss_func_setnet,
        device=device
    )
    
    all_train_losses_set.append(train_loss)
    all_val_losses_set.append(val_loss)
    all_test_losses_set.append(test_loss)
    
    all_train_correct_set.append(train_correct)
    all_val_correct_set.append(val_correct)
    all_test_correct_set.append(test_correct)
    
    all_train_results_set.append(train_results)
    all_val_results_set.append(val_results)
    all_test_results_set.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

711 (78%)]	Total Loss: 4.6158	Avg Loss: 0.0001
Train Epoch: 48	[93000/117711 (79%)]	Total Loss: 4.6641	Avg Loss: 0.0001
Train Epoch: 48	[94000/117711 (80%)]	Total Loss: 4.7153	Avg Loss: 0.0001
Train Epoch: 48	[95000/117711 (81%)]	Total Loss: 4.7613	Avg Loss: 0.0001
Train Epoch: 48	[96000/117711 (81%)]	Total Loss: 4.8099	Avg Loss: 0.0001
Train Epoch: 48	[97000/117711 (82%)]	Total Loss: 4.8621	Avg Loss: 0.0001
Train Epoch: 48	[98000/117711 (83%)]	Total Loss: 4.9163	Avg Loss: 0.0001
Train Epoch: 48	[99000/117711 (84%)]	Total Loss: 4.9741	Avg Loss: 0.0001
Train Epoch: 48	[100000/117711 (85%)]	Total Loss: 5.0280	Avg Loss: 0.0001
Train Epoch: 48	[101000/117711 (86%)]	Total Loss: 5.0757	Avg Loss: 0.0001
Train Epoch: 48	[102000/117711 (87%)]	Total Loss: 5.1473	Avg Loss: 0.0001
Train Epoch: 48	[103000/117711 (87%)]	Total Loss: 5.2000	Avg Loss: 0.0001
Train Epoch: 48	[104000/117711 (88%)]	Total Loss: 5.2608	Avg Loss: 0.0001
Train Epoch: 48	[105000/117711 (89%)]	Total Loss: 5.3027	Avg Loss: 0.000

In [10]:
# SAVE
models_folder = 'saved_models'
model_name_fx = '20200924_fxnet'
model_name_set = '20200924_setnet'
results_folder = 'saved_results'
results_subfolder = model_name_fx + '_and_setnet'

In [11]:
# SAVE MODEL
torch.save(fxnet, '%s/%s' % (models_folder, model_name_fx))
torch.save(setnet, '%s/%s' % (models_folder, model_name_set))

In [15]:
# SAVE RESULTS
all_train_losses_npy_fx = np.array(all_train_losses_fx)
all_train_correct_npy_fx = np.array(all_train_correct_fx)
all_train_results_npy_fx = np.array(all_train_results_fx)

all_val_losses_npy_fx = np.array(all_val_losses_fx)
all_val_correct_npy_fx = np.array(all_val_correct_fx)
all_val_results_npy_fx = np.array(all_val_results_fx)

all_test_losses_npy_fx = np.array(all_test_losses_fx)
all_test_correct_npy_fx = np.array(all_test_correct_fx)
all_test_results_npy_fx = np.array(all_test_results_fx)

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_losses_fx')), arr=all_train_losses_npy_fx)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_correct_fx')), arr=all_train_correct_npy_fx)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_results_fx')), arr=all_train_results_npy_fx)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_losses_fx')), arr=all_val_losses_npy_fx)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_correct_fx')), arr=all_val_correct_npy_fx)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_results_fx')), arr=all_val_results_npy_fx)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_losses_fx')), arr=all_test_losses_npy_fx)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_correct_fx')), arr=all_test_correct_npy_fx)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_results_fx')), arr=all_test_results_npy_fx)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'fx_labels')), arr=fx_labels_npy)

all_train_losses_npy_set = np.array(all_train_losses_set)
all_train_correct_npy_set = np.array(all_train_correct_set)
all_train_results_npy_set = np.array(all_train_results_set)

all_val_losses_npy_set = np.array(all_val_losses_set)
all_val_correct_npy_set = np.array(all_val_correct_set)
all_val_results_npy_set = np.array(all_val_results_set)

all_test_losses_npy_set = np.array(all_test_losses_set)
all_test_correct_npy_set = np.array(all_test_correct_set)
all_test_results_npy_set = np.array(all_test_results_set)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_losses_set')), arr=all_train_losses_npy_set)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_correct_set')), arr=all_train_correct_npy_set)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_results_set')), arr=all_train_results_npy_set)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_losses_set')), arr=all_val_losses_npy_set)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_correct_set')), arr=all_val_correct_npy_set)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_results_set')), arr=all_val_results_npy_set)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_losses_set')), arr=all_test_losses_npy_set)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_correct_set')), arr=all_test_correct_npy_set)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_results_set')), arr=all_test_results_npy_set)